# Network Analysis with Emissions

## Example from social circles data


### Data Description

https://snap.stanford.edu/data/ego-Facebook.html

This dataset (*facebook_combined.txt*) consists of circles from Facebook (4,039 nodes and 88,234 edges) 

### Load required libraries

In [4]:
import networkx as nx
import scipy.sparse as sp
import psutil, time, os, gc, statistics, warnings
import numpy as np
import pandas as pd
from codecarbon import EmissionsTracker

warnings.filterwarnings('ignore')

### Functions

In [5]:
# Function to get memory usage
def get_memory_usage():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss / 1024 / 1024  # Memory in MB

# Function to run analysis with multiple iterations
def run_analysis_with_repeats(analysis_func, file_path, num_runs=5):
    results = []
    
    for _ in range(num_runs):
        gc.collect() # clear memory before each run
        result = analysis_func(file_path)
        results.append(result)
    
    # Compute statistics
    metrics = {
        "Method": results[0]["Method"],
        "Nodes": results[0]["Nodes"],
        "Edges": results[0]["Edges"],
        "Avg Degree": results[0]["Avg Degree"],
        "Density": results[0]["Density"],
        "Time (s)": statistics.mean([r["Time (s)"] for r in results]),
        "Time Std (s)": statistics.stdev([r["Time (s)"] for r in results]) if num_runs > 1 else 0,
        "Memory (MB)": statistics.mean([r["Memory (MB)"] for r in results]),
        "Memory Std (MB)": statistics.stdev([r["Memory (MB)"] for r in results]) if num_runs > 1 else 0,
        "Emissions (kg CO2eq)": statistics.mean([r["Emissions (kg CO2eq)"] for r in results]),
        "Emissions Std (kg CO2eq)": statistics.stdev([r["Emissions (kg CO2eq)"] for r in results]) if num_runs > 1 else 0
    }
    
    return metrics, results[0]["Top 20 Nodes"]  # Return metrics and top nodes from first run

# Function to analyze full graph with NetworkX
def analyze_networkx(file_path):
    start_time = time.time()
    tracker = EmissionsTracker()
    tracker.start()
    start_memory = get_memory_usage()
    
    G = nx.read_edgelist(file_path, nodetype=int, create_using=nx.Graph())
    
    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()
    avg_degree = sum(dict(G.degree()).values()) / num_nodes
    density = (2 * num_edges) / (num_nodes * (num_nodes - 1)) if num_nodes > 1 else 0
    
    # Find top 20 nodes by degree
    degrees = G.degree()
    top_nodes = sorted(degrees, key=lambda x: x[1], reverse=True)[:20]
    top_nodes_dict = [{"Node": node, "Degree": degree} for node, degree in top_nodes]
    
    exec_time = time.time() - start_time
    memory_used = get_memory_usage() - start_memory
    emissions = tracker.stop()
    
    print(f"Nodes: {num_nodes}")
    print(f"Edges: {num_edges}")
    print(f"Average Degree: {avg_degree:,.2f}")
    print(f"Density: {density:,.6f}")
    print(f"Execution Time: {exec_time:,.2f} seconds")
    print(f"Memory Used: {memory_used:,.2f} MB")
    print(f"Carbon Emissions: {emissions:,.6f} kg CO2eq")
    print("Top 20 Most Connected Nodes:")
    for node in top_nodes_dict:
        print(f"  Node {node['Node']}: Degree {node['Degree']}")
    
    return {
        "Method": "NetworkX (Full)",
        "Nodes": num_nodes,
        "Edges": num_edges,
        "Avg Degree": avg_degree,
        "Density": density,
        "Time (s)": exec_time,
        "Memory (MB)": memory_used,
        "Emissions (kg CO2eq)": emissions,
        "Top 20 Nodes": top_nodes_dict
    }

# SciPy sparse matrix analysis
def analyze_scipy_sparse(file_path):
    start_time = time.time()   
    tracker = EmissionsTracker()
    tracker.start()
    start_memory = get_memory_usage()
    
    # Load edge list with pandas for better performance
    edges_df = pd.read_csv(file_path, sep=' ', header=None, dtype=np.int32, engine='c')
    edges = edges_df.to_numpy()
    
    # Vectorized node mapping
    nodes, inverse_indices = np.unique(edges, return_inverse=True)
    num_nodes = len(nodes)
    edge_indices = inverse_indices.reshape(edges.shape)  # Shape: (m, 2)
    
    # Create row and column arrays for symmetric adjacency matrix
    rows = np.concatenate([edge_indices[:, 0], edge_indices[:, 1]])
    cols = np.concatenate([edge_indices[:, 1], edge_indices[:, 0]])
    data = np.ones(len(rows), dtype=np.int32)
    
    adj_matrix = sp.csr_matrix((data, (rows, cols)), shape=(num_nodes, num_nodes))
    
    num_edges = adj_matrix.nnz // 2
    degrees = np.array(adj_matrix.sum(axis=1)).flatten()
    avg_degree = degrees.mean()
    density = (2 * num_edges) / (num_nodes * (num_nodes - 1)) if num_nodes > 1 else 0
    
    # Find top 20 nodes by degree
    top_indices = np.argpartition(degrees, -20)[-20:]
    top_degrees = degrees[top_indices]
    top_nodes = [(nodes[i], degrees[i]) for i in top_indices]
    top_nodes = sorted(top_nodes, key=lambda x: x[1], reverse=True)[:20]
    top_nodes_dict = [{"Node": node, "Degree": degree} for node, degree in top_nodes]
    
    exec_time = time.time() - start_time
    memory_used = get_memory_usage() - start_memory
    emissions = tracker.stop()
   
    
    print(f"Nodes: {num_nodes}")
    print(f"Edges: {num_edges}")
    print(f"Average Degree: {avg_degree:,.2f}")
    print(f"Density: {density:,.6f}")
    print(f"Execution Time: {exec_time:,.2f} seconds")
    print(f"Memory Used: {memory_used:,.2f} MB")
    print(f"Carbon Emissions: {emissions:,.6f} kg CO2eq")
    print("Top 20 Most Connected Nodes:")
    for node in top_nodes_dict:
        print(f"  Node {node['Node']}: Degree {node['Degree']}")
    
    return {
        "Method": "SciPy Sparse (Full)",
        "Nodes": num_nodes,
        "Edges": num_edges,
        "Avg Degree": avg_degree,
        "Density": density,
        "Time (s)": exec_time,
        "Memory (MB)": memory_used,
        "Emissions (kg CO2eq)": emissions,
        "Top 20 Nodes": top_nodes_dict
    }


### Main function and combined output

In [ ]:
# Main execution
def main():
    file_path = "data/facebook_combined.txt"
    
    if not os.path.exists(file_path):
        print(f"Error: {file_path} not found. Please download from http://snap.stanford.edu/data/egnets-Facebook.html")
        return
    
    num_runs = 50
    results = []
    top_nodes_dfs = []
    
    # List of analysis functions to loop over
    analysis_functions = [
        analyze_networkx,
        analyze_scipy_sparse
    ]
    
    # Run each analysis function in a loop
    for analysis_func in analysis_functions:
        print(f"\nRunning {analysis_func.__name__} analysis ({num_runs} runs)")
        metrics, top_nodes = run_analysis_with_repeats(analysis_func, file_path, num_runs)
        results.append(metrics)
        top_nodes_df = pd.DataFrame(top_nodes)
        top_nodes_df['Method'] = metrics["Method"]
        top_nodes_dfs.append(top_nodes_df)

    # Combine results
    print("\nTable: Performance Comparison (Averaged over runs)")
    df = pd.DataFrame(results)
    print(df.round(6))
    df.to_csv("01-02-graph_analysis_comparison.csv", index=False)
    print("\nResults saved to 01-02-graph_analysis_comparison.csv")
    
    # Save top nodes to CSV
    top_nodes_combined = pd.concat(top_nodes_dfs, ignore_index=True)
    top_nodes_combined.to_csv("01-02-top_20_nodes.csv", index=False)
    print("Top 20 nodes saved to 01-02-top_20_nodes.csv")

if __name__ == "__main__":
    main()

[codecarbon WARNING @ 22:08:52] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon WARNING @ 22:08:52] Error while trying to count physical CPUs: [Errno 2] No such file or directory: 'lscpu'. Defaulting to 1.
[codecarbon INFO @ 22:08:52] [setup] RAM Tracking...
[codecarbon INFO @ 22:08:52] [setup] CPU Tracking...
[codecarbon WARNING @ 22:08:53] We saw that you have a Apple M4 but we don't know it. Please contact us.



Running NetworkX analysis (50 runs)


[codecarbon WARNING @ 22:08:53] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Mac OS detected: Please install Intel Power Gadget or enable PowerMetrics sudo to measure CPU

[codecarbon INFO @ 22:08:53] CPU Model on constant consumption mode: Apple M4
[codecarbon WARNING @ 22:08:53] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 22:08:53] [setup] GPU Tracking...
[codecarbon INFO @ 22:08:53] No GPU found.
[codecarbon INFO @ 22:08:53] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: Unspecified
            
[codecarbon INFO @ 22:08:53] >>> Tracker's metadata:
[codecarbon INFO @ 22:08:53]   Platform system: macOS-15.5-arm64-arm-64bit-Mach-O
[codecarbon INFO @ 22:08:53]   Python version: 3.13.5
[codecarbon INFO @ 22:08:53]   CodeCarbon version: 3.0.2
[codecarbon INFO @ 22:08:5

Nodes: 4039
Edges: 88234
Average Degree: 43.69
Density: 0.010820
Execution Time: 0.48 seconds
Memory Used: 12.28 MB
Carbon Emissions: 0.000000 kg CO2eq
Top 20 Most Connected Nodes:
  Node 107: Degree 1045
  Node 1684: Degree 792
  Node 1912: Degree 755
  Node 3437: Degree 547
  Node 0: Degree 347
  Node 2543: Degree 294
  Node 2347: Degree 291
  Node 1888: Degree 254
  Node 1800: Degree 245
  Node 1663: Degree 235
  Node 1352: Degree 234
  Node 2266: Degree 234
  Node 483: Degree 231
  Node 348: Degree 229
  Node 1730: Degree 226
  Node 1985: Degree 224
  Node 1941: Degree 223
  Node 2233: Degree 222
  Node 2142: Degree 221
  Node 1431: Degree 220


[codecarbon WARNING @ 22:08:53] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Mac OS detected: Please install Intel Power Gadget or enable PowerMetrics sudo to measure CPU

[codecarbon INFO @ 22:08:53] CPU Model on constant consumption mode: Apple M4
[codecarbon WARNING @ 22:08:53] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 22:08:53] [setup] GPU Tracking...
[codecarbon INFO @ 22:08:53] No GPU found.
[codecarbon INFO @ 22:08:53] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: Unspecified
            
[codecarbon INFO @ 22:08:53] >>> Tracker's metadata:
[codecarbon INFO @ 22:08:53]   Platform system: macOS-15.5-arm64-arm-64bit-Mach-O
[codecarbon INFO @ 22:08:53]   Python version: 3.13.5
[codecarbon INFO @ 22:08:53]   CodeCarbon version: 3.0.2
[codecarbon INFO @ 22:08:5

...

Nodes: 4039
Edges: 88234
Average Degree: 43.69
Density: 0.010820
Execution Time: 0.43 seconds
Memory Used: 1.83 MB
Carbon Emissions: 0.000000 kg CO2eq
Top 20 Most Connected Nodes:
  Node 107: Degree 1045
  Node 1684: Degree 792
  Node 1912: Degree 755
  Node 3437: Degree 547
  Node 0: Degree 347
  Node 2543: Degree 294
  Node 2347: Degree 291
  Node 1888: Degree 254
  Node 1800: Degree 245
  Node 1663: Degree 235
  Node 1352: Degree 234
  Node 2266: Degree 234
  Node 483: Degree 231
  Node 348: Degree 229
  Node 1730: Degree 226
  Node 1985: Degree 224
  Node 1941: Degree 223
  Node 2233: Degree 222
  Node 2142: Degree 221
  Node 1431: Degree 220


[codecarbon WARNING @ 22:09:53] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Mac OS detected: Please install Intel Power Gadget or enable PowerMetrics sudo to measure CPU

[codecarbon INFO @ 22:09:53] CPU Model on constant consumption mode: Apple M4
[codecarbon WARNING @ 22:09:53] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 22:09:53] [setup] GPU Tracking...
[codecarbon INFO @ 22:09:53] No GPU found.
[codecarbon INFO @ 22:09:53] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: Unspecified
            
[codecarbon INFO @ 22:09:53] >>> Tracker's metadata:
[codecarbon INFO @ 22:09:53]   Platform system: macOS-15.5-arm64-arm-64bit-Mach-O
[codecarbon INFO @ 22:09:53]   Python version: 3.13.5
[codecarbon INFO @ 22:09:53]   CodeCarbon version: 3.0.2
[codecarbon INFO @ 22:09:5

Nodes: 4039
Edges: 88234
Average Degree: 43.69
Density: 0.010820
Execution Time: 0.43 seconds
Memory Used: 2.70 MB
Carbon Emissions: 0.000000 kg CO2eq
Top 20 Most Connected Nodes:
  Node 107: Degree 1045
  Node 1684: Degree 792
  Node 1912: Degree 755
  Node 3437: Degree 547
  Node 0: Degree 347
  Node 2543: Degree 294
  Node 2347: Degree 291
  Node 1888: Degree 254
  Node 1800: Degree 245
  Node 1663: Degree 235
  Node 1352: Degree 234
  Node 2266: Degree 234
  Node 483: Degree 231
  Node 348: Degree 229
  Node 1730: Degree 226
  Node 1985: Degree 224
  Node 1941: Degree 223
  Node 2233: Degree 222
  Node 2142: Degree 221
  Node 1431: Degree 220


[codecarbon WARNING @ 22:09:54] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Mac OS detected: Please install Intel Power Gadget or enable PowerMetrics sudo to measure CPU

[codecarbon INFO @ 22:09:54] CPU Model on constant consumption mode: Apple M4
[codecarbon WARNING @ 22:09:54] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 22:09:54] [setup] GPU Tracking...
[codecarbon INFO @ 22:09:54] No GPU found.
[codecarbon INFO @ 22:09:54] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: Unspecified
            
[codecarbon INFO @ 22:09:54] >>> Tracker's metadata:
[codecarbon INFO @ 22:09:54]   Platform system: macOS-15.5-arm64-arm-64bit-Mach-O
[codecarbon INFO @ 22:09:54]   Python version: 3.13.5
[codecarbon INFO @ 22:09:54]   CodeCarbon version: 3.0.2
[codecarbon INFO @ 22:09:5

Nodes: 4039
Edges: 88234
Average Degree: 43.69
Density: 0.010820
Execution Time: 0.42 seconds
Memory Used: 0.61 MB
Carbon Emissions: 0.000000 kg CO2eq
Top 20 Most Connected Nodes:
  Node 107: Degree 1045
  Node 1684: Degree 792
  Node 1912: Degree 755
  Node 3437: Degree 547
  Node 0: Degree 347
  Node 2543: Degree 294
  Node 2347: Degree 291
  Node 1888: Degree 254
  Node 1800: Degree 245
  Node 1663: Degree 235
  Node 1352: Degree 234
  Node 2266: Degree 234
  Node 483: Degree 231
  Node 348: Degree 229
  Node 1730: Degree 226
  Node 1985: Degree 224
  Node 1941: Degree 223
  Node 2233: Degree 222
  Node 2142: Degree 221
  Node 1431: Degree 220

Table: Performance Comparison (Averaged over runs)
                Method  Nodes  Edges  Avg Degree  Density  Time (s)  \
0      NetworkX (Full)   4039  88234   43.691013  0.01082  0.590340   
1  SciPy Sparse (Full)   4039  88234   43.691013  0.01082  0.534191   

   Time Std (s)  Memory (MB)  Memory Std (MB)  Emissions (kg CO2eq)  \
0      0

## Reference

1. 